In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import LSTM
from keras.layers import Dropout

In [23]:
all_mines = pd.read_csv('/Users/taddbackus/School/Capstone/Data/earthEngine/new/all_mines_no_filter_with_precip.csv')
print(all_mines.columns)

Index(['Unnamed: 0', 'Company', 'start_date', 'Observations', 'orig_perm_id',
       'mine_id', 'State', 'water', 'trees', 'grass', 'flooded_vegetation',
       'crops', 'shrub_and_scrub', 'built', 'bare', 'snow_and_ice', 'company',
       'coalmine_op_status', 'mine_name', 'permit_id', 'national_id',
       'coal_bed_names', 'inspectable_unit_status', 'post_smcra',
       'reported_area', 'permit_application_type', 'area_mine', 'contour',
       'mountaintop', 'steep_slope', 'highwall', 'auger', 'contact',
       'Number_company_to_permit_id', 'edit_month', 'edit_year',
       'permit_weekday', 'permit_approval_month', 'permit_approval_year',
       'permit_application_year', 'WaterQuality', 'Max Temperature',
       'Precipitation'],
      dtype='object')


/Users/taddbackus/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3505: DtypeWarning: Columns (6,16,18,19,20,21,35,36,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [42]:
all_mines['uniqueID'] = all_mines.groupby(['mine_id','orig_perm_id','Company']).ngroup()
date_to_sequence = {'2017-07-01':0,
                    '2019-07-01':1,
                    '2021-07-01':2,
                    '2023-07-01':3}
all_mines['Sequence'] = all_mines['start_date'].map(date_to_sequence)
all_mines.sort_values(by=['uniqueID','Sequence'],inplace=True)
print(all_mines.head(16))

       Unnamed: 0                           Company  start_date  Observations  \
25101       25101               LEVISA ENERGIES INC  2017-07-01      1.000000   
9943         9943               LEVISA ENERGIES INC  2019-07-01      3.000000   
6644         6644               LEVISA ENERGIES INC  2021-07-01      2.000000   
263           263               LEVISA ENERGIES INC  2023-07-01      2.000000   
30008       30008      ERP ENVIRONMENTAL FUND, INC.  2019-07-01     15.000000   
28614       28614      ERP ENVIRONMENTAL FUND, INC.  2021-07-01     36.000000   
27587       27587      ERP ENVIRONMENTAL FUND, INC.  2023-07-01      2.000000   
30864       30864  PREMIUM ELKHORN COAL CORPORATION  2017-07-01      2.333333   
11996       11996  PREMIUM ELKHORN COAL CORPORATION  2019-07-01      3.000000   
8685         8685  PREMIUM ELKHORN COAL CORPORATION  2021-07-01      2.000000   
2349         2349  PREMIUM ELKHORN COAL CORPORATION  2023-07-01      2.000000   
30862       30862      DALCO

In [44]:
columns_to_drop = ['Unnamed: 0', 'company']
all_mines = all_mines.drop(columns = columns_to_drop)


all_mines.columns

Index(['Company', 'start_date', 'Observations', 'orig_perm_id', 'mine_id',
       'State', 'water', 'trees', 'grass', 'flooded_vegetation', 'crops',
       'shrub_and_scrub', 'built', 'bare', 'snow_and_ice',
       'coalmine_op_status', 'mine_name', 'permit_id', 'national_id',
       'coal_bed_names', 'inspectable_unit_status', 'post_smcra',
       'reported_area', 'permit_application_type', 'area_mine', 'contour',
       'mountaintop', 'steep_slope', 'highwall', 'auger', 'contact',
       'Number_company_to_permit_id', 'edit_month', 'edit_year',
       'permit_weekday', 'permit_approval_month', 'permit_approval_year',
       'permit_application_year', 'WaterQuality', 'Max Temperature',
       'Precipitation', 'uniqueID', 'Sequence'],
      dtype='object')

In [45]:
all_mines

,Company,start_date,Observations,orig_perm_id,mine_id,State,water,trees,grass,flooded_vegetation,...,edit_year,permit_weekday,permit_approval_month,permit_approval_year,permit_application_year,WaterQuality,Max Temperature,Precipitation,uniqueID,Sequence
25101,LEVISA ENERGIES INC,2017-07-01,1.000000,0365205,00000000000000000000,ky,0.031829,0.664179,0.063150,0.027551,...,1999,Saturday,12.0,1899,NaT,-0.336781,300.366659,13.333333,0,0
9943,LEVISA ENERGIES INC,2019-07-01,3.000000,0365205,00000000000000000000,ky,0.032339,0.718107,0.045966,0.030282,...,1999.0,Saturday,12.0,1899,NaT,-0.508053,304.399994,0.000000,0,1
6644,LEVISA ENERGIES INC,2021-07-01,2.000000,0365205,00000000000000000000,ky,0.032204,0.733225,0.041907,0.031283,...,1999.0,Saturday,12.0,1899,NaT,-0.736694,297.399994,30.799999,0,2
263,LEVISA ENERGIES INC,2023-07-01,2.000000,0365205,00000000000000000000,ky,0.032364,0.742873,0.034686,0.030389,...,1999.0,Saturday,12.0,1899,NaT,0.044010,299.299988,9.200000,0,3
30008,"ERP ENVIRONMENTAL FUND, INC.",2019-07-01,15.000000,S010380,00000000000000000000,wv,0.032505,0.733745,0.034282,0.029633,...,2023,Tuesday,10.0,1980,NaT,NaN,NaN,NaN,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3385,BRANHAM & BAKER COAL COMPANY INC,2023-07-01,5.000000,0365200,0000000000000000313c,ky,0.031722,0.740254,0.032421,0.027490,...,1999.0,Saturday,12.0,1899,NaT,-0.077258,298.700012,9.300000,10219,3
38317,BRANHAM & BAKER COAL COMPANY INC,2017-07-01,2.666667,0365204,0000000000000000313d,NaN,0.033089,0.736873,0.034586,0.029194,...,NaN,NaN,NaN,NaN,NaN,-0.606016,299.900004,12.933334,10220,0
17696,BRANHAM & BAKER COAL COMPANY INC,2019-07-01,4.000000,0365204,0000000000000000313d,ky,0.032632,0.739368,0.034791,0.028231,...,1999,Saturday,12.0,1899,NaT,-0.578522,304.100006,0.000000,10220,1
22644,BRANHAM & BAKER COAL COMPANY INC,2021-07-01,2.000000,0365204,0000000000000000313d,ky,0.033355,0.734218,0.034664,0.029445,...,1999,Saturday,12.0,1899,NaT,-0.743660,296.700012,29.700001,10220,2


In [46]:
all_mines.dtypes

Company                         object
start_date                      object
Observations                   float64
orig_perm_id                    object
mine_id                         object
State                           object
water                          float64
trees                          float64
grass                          float64
flooded_vegetation             float64
crops                          float64
shrub_and_scrub                float64
built                          float64
bare                           float64
snow_and_ice                   float64
coalmine_op_status             float64
mine_name                       object
permit_id                       object
national_id                     object
coal_bed_names                  object
inspectable_unit_status        float64
post_smcra                     float64
reported_area                  float64
permit_application_type        float64
area_mine                      float64
contour                  

In [47]:
for column_name, dtype in all_mines.dtypes.items():
    if dtype == 'object':
        column_values = all_mines[column_name]
        
        num_unique_values = column_values.nunique()
        
        print(f'Column: {column_name}')
        print(f'Number of Unique Values: {num_unique_values}\n')
        value_counts = column_values.value_counts()
        print(value_counts)
        print('-------------------------------------------\n')

Column: Company
Number of Unique Values: 3065

REVELATION ENERGY LLC                525
MARTIN COUNTY COAL CORPORATION       389
LCC KENTUCKY LLC                     362
NALLY & HAMILTON ENTERPRISES INC     320
ICG HAZARD LLC                       318
                                    ... 
CRADDOCK & SON COAL COMPANY, INC.      1
ILLUSION COAL COMPANY, INC.            1
JACK RUN COAL CORPORATION, INC.        1
PERRY AND HYLTON, INCORPORATED         1
ELMINA COAL COMPANY                    1
Name: Company, Length: 3065, dtype: int64
-------------------------------------------

Column: start_date
Number of Unique Values: 4

2019-07-01    9995
2021-07-01    9702
2017-07-01    9597
2023-07-01    9368
Name: start_date, dtype: int64
-------------------------------------------

Column: orig_perm_id
Number of Unique Values: 8860

8980437    24
0130182    24
0268001    24
2667000    24
0070128    24
           ..
U005483     1
U004783     1
D001481     1
U201689     1
S200498     1
Name: orig

In [48]:
columns_to_encode = ['Company', 'coal_bed_names', 
                     'State', 'permit_weekday', 'permit_approval_year']

# columns_to_encode = ['Company', 'mine_name', 'coal_bed_names', 
#                      'State', 'permit_weekday', 'permit_approval_year']

all_mines_encoded = pd.get_dummies(all_mines, columns = columns_to_encode)

all_mines_encoded

,start_date,Observations,orig_perm_id,mine_id,water,trees,grass,flooded_vegetation,crops,shrub_and_scrub,...,permit_approval_year_2010,permit_approval_year_2011,permit_approval_year_2012,permit_approval_year_2013,permit_approval_year_2014,permit_approval_year_2015,permit_approval_year_2016,permit_approval_year_2017,permit_approval_year_2018,permit_approval_year_NaT
25101,2017-07-01,1.000000,0365205,00000000000000000000,0.031829,0.664179,0.063150,0.027551,0.050951,0.041291,...,0,0,0,0,0,0,0,0,0,0
9943,2019-07-01,3.000000,0365205,00000000000000000000,0.032339,0.718107,0.045966,0.030282,0.037086,0.036018,...,0,0,0,0,0,0,0,0,0,0
6644,2021-07-01,2.000000,0365205,00000000000000000000,0.032204,0.733225,0.041907,0.031283,0.036091,0.031766,...,0,0,0,0,0,0,0,0,0,0
263,2023-07-01,2.000000,0365205,00000000000000000000,0.032364,0.742873,0.034686,0.030389,0.032345,0.031807,...,0,0,0,0,0,0,0,0,0,0
30008,2019-07-01,15.000000,S010380,00000000000000000000,0.032505,0.733745,0.034282,0.029633,0.032019,0.035532,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3385,2023-07-01,5.000000,0365200,0000000000000000313c,0.031722,0.740254,0.032421,0.027490,0.029960,0.037491,...,0,0,0,0,0,0,0,0,0,0
38317,2017-07-01,2.666667,0365204,0000000000000000313d,0.033089,0.736873,0.034586,0.029194,0.032866,0.032823,...,0,0,0,0,0,0,0,0,0,0
17696,2019-07-01,4.000000,0365204,0000000000000000313d,0.032632,0.739368,0.034791,0.028231,0.033070,0.032641,...,0,0,0,0,0,0,0,0,0,0
22644,2021-07-01,2.000000,0365204,0000000000000000313d,0.033355,0.734218,0.034664,0.029445,0.033263,0.033359,...,0,0,0,0,0,0,0,0,0,0


In [49]:
for column_name in all_mines_encoded.columns:
    if all_mines_encoded[column_name].dtype == 'int64':
        all_mines_encoded[column_name] = all_mines_encoded[column_name].astype('float32')
        
all_mines_encoded

,start_date,Observations,orig_perm_id,mine_id,water,trees,grass,flooded_vegetation,crops,shrub_and_scrub,...,permit_approval_year_2010,permit_approval_year_2011,permit_approval_year_2012,permit_approval_year_2013,permit_approval_year_2014,permit_approval_year_2015,permit_approval_year_2016,permit_approval_year_2017,permit_approval_year_2018,permit_approval_year_NaT
25101,2017-07-01,1.000000,0365205,00000000000000000000,0.031829,0.664179,0.063150,0.027551,0.050951,0.041291,...,0,0,0,0,0,0,0,0,0,0
9943,2019-07-01,3.000000,0365205,00000000000000000000,0.032339,0.718107,0.045966,0.030282,0.037086,0.036018,...,0,0,0,0,0,0,0,0,0,0
6644,2021-07-01,2.000000,0365205,00000000000000000000,0.032204,0.733225,0.041907,0.031283,0.036091,0.031766,...,0,0,0,0,0,0,0,0,0,0
263,2023-07-01,2.000000,0365205,00000000000000000000,0.032364,0.742873,0.034686,0.030389,0.032345,0.031807,...,0,0,0,0,0,0,0,0,0,0
30008,2019-07-01,15.000000,S010380,00000000000000000000,0.032505,0.733745,0.034282,0.029633,0.032019,0.035532,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3385,2023-07-01,5.000000,0365200,0000000000000000313c,0.031722,0.740254,0.032421,0.027490,0.029960,0.037491,...,0,0,0,0,0,0,0,0,0,0
38317,2017-07-01,2.666667,0365204,0000000000000000313d,0.033089,0.736873,0.034586,0.029194,0.032866,0.032823,...,0,0,0,0,0,0,0,0,0,0
17696,2019-07-01,4.000000,0365204,0000000000000000313d,0.032632,0.739368,0.034791,0.028231,0.033070,0.032641,...,0,0,0,0,0,0,0,0,0,0
22644,2021-07-01,2.000000,0365204,0000000000000000313d,0.033355,0.734218,0.034664,0.029445,0.033263,0.033359,...,0,0,0,0,0,0,0,0,0,0


In [50]:
del all_mines

In [51]:
object_columns = all_mines_encoded.select_dtypes(include = ['object'])

all_mines_encoded = all_mines_encoded.drop(columns = object_columns.columns)

all_mines_encoded

,Observations,water,trees,grass,flooded_vegetation,crops,shrub_and_scrub,built,bare,snow_and_ice,...,permit_approval_year_2010,permit_approval_year_2011,permit_approval_year_2012,permit_approval_year_2013,permit_approval_year_2014,permit_approval_year_2015,permit_approval_year_2016,permit_approval_year_2017,permit_approval_year_2018,permit_approval_year_NaT
25101,1.000000,0.031829,0.664179,0.063150,0.027551,0.050951,0.041291,0.032833,0.031638,0.034973,...,0,0,0,0,0,0,0,0,0,0
9943,3.000000,0.032339,0.718107,0.045966,0.030282,0.037086,0.036018,0.031099,0.032358,0.033141,...,0,0,0,0,0,0,0,0,0,0
6644,2.000000,0.032204,0.733225,0.041907,0.031283,0.036091,0.031766,0.028014,0.031360,0.032125,...,0,0,0,0,0,0,0,0,0,0
263,2.000000,0.032364,0.742873,0.034686,0.030389,0.032345,0.031807,0.029251,0.032672,0.032876,...,0,0,0,0,0,0,0,0,0,0
30008,15.000000,0.032505,0.733745,0.034282,0.029633,0.032019,0.035532,0.031059,0.036299,0.034627,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3385,5.000000,0.031722,0.740254,0.032421,0.027490,0.029960,0.037491,0.029695,0.037373,0.033526,...,0,0,0,0,0,0,0,0,0,0
38317,2.666667,0.033089,0.736873,0.034586,0.029194,0.032866,0.032823,0.031863,0.034403,0.034084,...,0,0,0,0,0,0,0,0,0,0
17696,4.000000,0.032632,0.739368,0.034791,0.028231,0.033070,0.032641,0.032580,0.033082,0.033378,...,0,0,0,0,0,0,0,0,0,0
22644,2.000000,0.033355,0.734218,0.034664,0.029445,0.033263,0.033359,0.032392,0.034801,0.034244,...,0,0,0,0,0,0,0,0,0,0


In [52]:
all_mines_encoded = all_mines_encoded.replace({True: 1, False: 0})

all_mines_encoded

,Observations,water,trees,grass,flooded_vegetation,crops,shrub_and_scrub,built,bare,snow_and_ice,...,permit_approval_year_2010,permit_approval_year_2011,permit_approval_year_2012,permit_approval_year_2013,permit_approval_year_2014,permit_approval_year_2015,permit_approval_year_2016,permit_approval_year_2017,permit_approval_year_2018,permit_approval_year_NaT
25101,1.000000,0.031829,0.664179,0.063150,0.027551,0.050951,0.041291,0.032833,0.031638,0.034973,...,0,0,0,0,0,0,0,0,0,0
9943,3.000000,0.032339,0.718107,0.045966,0.030282,0.037086,0.036018,0.031099,0.032358,0.033141,...,0,0,0,0,0,0,0,0,0,0
6644,2.000000,0.032204,0.733225,0.041907,0.031283,0.036091,0.031766,0.028014,0.031360,0.032125,...,0,0,0,0,0,0,0,0,0,0
263,2.000000,0.032364,0.742873,0.034686,0.030389,0.032345,0.031807,0.029251,0.032672,0.032876,...,0,0,0,0,0,0,0,0,0,0
30008,15.000000,0.032505,0.733745,0.034282,0.029633,0.032019,0.035532,0.031059,0.036299,0.034627,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3385,5.000000,0.031722,0.740254,0.032421,0.027490,0.029960,0.037491,0.029695,0.037373,0.033526,...,0,0,0,0,0,0,0,0,0,0
38317,2.666667,0.033089,0.736873,0.034586,0.029194,0.032866,0.032823,0.031863,0.034403,0.034084,...,0,0,0,0,0,0,0,0,0,0
17696,4.000000,0.032632,0.739368,0.034791,0.028231,0.033070,0.032641,0.032580,0.033082,0.033378,...,0,0,0,0,0,0,0,0,0,0
22644,2.000000,0.033355,0.734218,0.034664,0.029445,0.033263,0.033359,0.032392,0.034801,0.034244,...,0,0,0,0,0,0,0,0,0,0


In [53]:
nan_count = all_mines_encoded.isna().sum()

nan_count_sorted = nan_count.sort_values(ascending = False)

print(nan_count_sorted.head(20))

reported_area                  38662
permit_application_type        38658
edit_month                     21620
permit_approval_month          17741
highwall                       17028
inspectable_unit_status        16524
mountaintop                    13740
area_mine                      13740
auger                          13740
steep_slope                    13740
contour                        13740
post_smcra                     11812
Number_company_to_permit_id    11804
contact                        11804
coalmine_op_status             11804
WaterQuality                    5413
Precipitation                   5410
Max Temperature                 5410
Company_TWIN RESOURCES LLC         0
Company_U S COAL INC               0
dtype: int64


In [54]:
missing_data = all_mines_encoded[['reported_area', 'permit_application_type', 'edit_month', 'permit_approval_month',
                                  'highwall', 'steep_slope', 'contour', 'post_smcra', 'Number_company_to_permit_id',
                                  'contact', 'coalmine_op_status', 'WaterQuality', 'Max Temperature', 'Precipitation'
                                 ]]



missing_data.head(25)

,reported_area,permit_application_type,edit_month,permit_approval_month,highwall,steep_slope,contour,post_smcra,Number_company_to_permit_id,contact,coalmine_op_status,WaterQuality,Max Temperature,Precipitation
25101,NaN,NaN,11.0,12.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,-0.336781,300.366659,13.333333
9943,NaN,NaN,11.0,12.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,-0.508053,304.399994,0.000000
6644,NaN,NaN,11.0,12.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,-0.736694,297.399994,30.799999
263,NaN,NaN,11.0,12.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,0.044010,299.299988,9.200000
30008,NaN,NaN,6.0,10.0,NaN,0.0,1.0,1.0,1.0,2.0,1.0,NaN,NaN,NaN
28614,NaN,NaN,6.0,10.0,NaN,0.0,1.0,1.0,1.0,2.0,1.0,NaN,NaN,NaN
27587,NaN,NaN,6.0,10.0,NaN,0.0,1.0,1.0,1.0,2.0,1.0,NaN,NaN,NaN
30864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.456124,299.766663,17.100000
11996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.679023,303.000000,0.000000
8685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.736681,297.299988,43.799999


In [55]:
# drop these columns 
all_mines_encoded_droptest = all_mines_encoded.drop(['permit_application_type','reported_area',
                                                     'edit_month', 'permit_approval_month'], axis = 1)

all_mines_encoded_droptest


# Impute with mode for these columns 
columns_to_impute_with_mode = ['highwall', 'steep_slope', 'contour', 'area_mine', 
                               'auger', 'mountaintop', 'inspectable_unit_status', 
                              'post_smcra', 'coalmine_op_status', 'Number_company_to_permit_id', 'contact']

columns_to_impute_with_mean = ['WaterQuality', 'Precipitation', 'Max Temperature']

all_mines_encoded_droptest[columns_to_impute_with_mode] = all_mines_encoded_droptest[columns_to_impute_with_mode].fillna(all_mines_encoded_droptest[columns_to_impute_with_mode].mode().iloc[0])

all_mines_encoded_droptest[columns_to_impute_with_mean] = all_mines_encoded_droptest[columns_to_impute_with_mean].fillna(all_mines_encoded_droptest[columns_to_impute_with_mean].mean().iloc[0])


nan_count = all_mines_encoded_droptest.isna().sum()

nan_count_sorted = nan_count.sort_values(ascending = False)

print(nan_count_sorted.head(12))

Observations                                  0
Company_TWIN STAR MINING INC                  0
Company_TRITON ENERGY COAL INC                0
Company_TROY BRANDENBURG                      0
Company_TROY FREEMAN                          0
Company_TROY STIDHAM COAL CO                  0
Company_TRYWAY COAL CO                        0
Company_TUG VALLEY COAL PROCESSING COMPANY    0
Company_TUNNEL RIDGE, LLC                     0
Company_TUSCALOOSA ENERGY CORPORATION         0
Company_TWILIGHT CONST CO                     0
Company_TWILIGHT ENERGY, LLC                  0
dtype: int64


In [61]:
columns_to_exclude = ['trees', 'water', 'snow_and_ice', 'bare', 'built', 'shrub_and_scrub', 'crops']

selected_columns = all_mines_encoded_droptest.columns[~all_mines_encoded_droptest.columns.isin(columns_to_exclude)]

features = all_mines_encoded_droptest[selected_columns]
target = all_mines_encoded_droptest['trees']

X, y = [], []

sequence_length = 4
num_features = 4332

for i in range(len(features) - sequence_length):
    X.append(features.iloc[i:i + sequence_length].values)
    y.append(target.iloc[i + sequence_length])

X = np.array(X)
y = np.array(y)

In [62]:
del all_mines_encoded

NameError: name 'all_mines_encoded' is not defined

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.2,
                                                    random_state = 0)

In [64]:
X.shape

(38658, 4, 4332)

In [59]:
model = Sequential()

In [71]:
model.add(LSTM(64, input_shape=(sequence_length, num_features), return_sequences=True,
          activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(units=32))
model.add(Dropout(0.2))
model.add(Dense(1))  # Output layer for regression

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50
967/967 [==============================] - 17s 14ms/step - loss: 0.0276 - val_loss: 0.0186
Epoch 2/50
967/967 [==============================] - 11s 11ms/step - loss: 0.0205 - val_loss: 0.0184
Epoch 3/50
967/967 [==============================] - 11s 11ms/step - loss: 0.0198 - val_loss: 0.0187
Epoch 4/50
967/967 [==============================] - 11s 11ms/step - loss: 0.0191 - val_loss: 0.0185
Epoch 5/50
967/967 [==============================] - 11s 11ms/step - loss: 0.0187 - val_loss: 0.0188
Epoch 6/50
967/967 [==============================] - 11s 11ms/step - loss: 0.0185 - val_loss: 0.0185
Epoch 7/50
967/967 [==============================] - 11s 11ms/step - loss: 0.0184 - val_loss: 0.0187
Epoch 8/50
967/967 [==============================] - 11s 11ms/step - loss: 0.0183 - val_loss: 0.0187
Epoch 9/50
967/967 [==============================] - 11s 11ms/step - loss: 0.0182 - val_loss: 0.0185
Epoch 10/50
967/967 [==============================] - 11s 11ms/step - loss: 0.018

In [72]:
y_pred = model.predict(X_test)
for i,j in zip(y_test,y_pred):
    print(i,'===',j)
    print('=============\n')

242/242 [==============================] - 2s 4ms/step
0.7320708949312879 === [0.64201224]

0.7424846105731967 === [0.64201224]

0.7380423764008339 === [0.64201224]

0.4497604970154094 === [0.64201224]

0.7397679513087496 === [0.64201224]

0.4412789834946971 === [0.64201224]

0.7311990039015654 === [0.64201224]

0.7468936452642083 === [0.64201224]

0.747973891440779 === [0.64201224]

0.1765112524386495 === [0.64201224]

0.6713337470708346 === [0.64201224]

0.7403022517516676 === [0.64201224]

0.6283341944683343 === [0.64201224]

0.7320880494166717 === [0.64201224]

0.7460714431945235 === [0.64201224]

0.7275996171714117 === [0.64201224]

0.6146651558810845 === [0.64201224]

0.7328038796808364 === [0.64201224]

0.7338037153757695 === [0.64201224]

0.7338791122892871 === [0.64201224]

0.7322398552981516 === [0.64201224]

0.2114517837762832 === [0.64201224]

0.7260910858106064 === [0.64201224]

0.7433633526864772 === [0.64201224]

0.4735588564071804 === [0.64201224]

0.230828667699825 ===

In [74]:
print(y_pred)

[[0.64201224]
 [0.64201224]
 [0.64201224]
 ...
 [0.64201224]
 [0.64201224]
 [0.64201224]]


In [73]:

mse = mean_squared_error(y_test,y_pred[0])
mae = mean_absolute_error(y_test,y_pred[0])
r2 = r2_score(y_test,y_pred[0])

print('\nMean Squared Error (MSE):', mse)
print('Mean Absolute Error (MAE):', mae)
print('R-squared (R^2):', r2)

ValueError: Found input variables with inconsistent numbers of samples: [7732, 1]